# Qwen2.5-7B模型微调流程
本notebook参考Qwen2_5_7B_Alpaca_fintune.ipynb，使用已处理好的QA数据进行微调。

## 2. 加载和配置Qwen2.5-7B模型
设置最大序列长度、数据类型和量化方式，加载预训练模型和分词器。

## 3. 添加LoRA适配器
为模型添加LoRA适配器，只微调部分参数以节省显存和加速训练。

## 4. 加载QA数据集
加载已处理好的QA数据集（qa_train.json），用于微调。

## 5. 格式化数据并添加EOS标记
定义格式化函数，将指令、输入和输出拼接为训练文本，并在末尾添加EOS标记。

## 6. 训练模型
使用TRL的SFTTrainer进行微调，设置训练参数如批次大小、学习率、训练步数等。

## 7. 显存信息监控
通过torch.cuda获取和打印显卡显存使用情况，便于监控资源消耗。

## 8. 启动训练并监控显存变化
开始训练模型，并显示训练结果和显存变化。

## 9. 推理与保存微调后的模型
训练完成后，进行简单推理验证，并保存LoRA适配器和分词器。

In [ ]:
import torch
from unsloth import FastLanguageModel
from trl import SFTConfig, SFTTrainer
from datasets import Dataset
import json
from sklearn.metrics import recall_score
import numpy as np

# ===================== 显存计算工具函数 =====================
def calculate_vram_usage(batch_size, seq_len):
    """计算FP16精度下的显存占用（适用于LoRA微调场景）"""
    model_weight = 14  # 7B模型FP16权重约14GB
    kv_cache = batch_size * seq_len * 4 * 2 / (1024**3)  # 4层注意力机制，FP16精度
    gradient_checkpoint = 0.6  # 梯度检查点节省40%显存
    return round((model_weight + kv_cache) * gradient_checkpoint, 2)


# ===================== 1. 金融场景训练配置（适配小样本） =====================
finance_sft_config = {
    "learning_rate": 2e-5,    # 小学习率适配小样本，避免过拟合
    "num_train_epochs": 8,    # 多轮训练充分学习有限样本
    "per_device_train_batch_size": 2,  # 小批量适应显存
    "gradient_accumulation_steps": 4,  # 累积梯度等效增大批量
    "lora_r": 16,             # 适配金融领域知识的LoRA秩
    "max_seq_length": 2048,   # 金融文本较长，保持合理序列长度
}

# 提前计算显存占用并提示
predicted_vram = calculate_vram_usage(
    finance_sft_config["per_device_train_batch_size"],
    finance_sft_config["max_seq_length"]
)
print(f"预估显存占用: {predicted_vram} GB")


# ===================== 2. 加载和配置Qwen2.5-7B模型 =====================
max_seq_length = finance_sft_config["max_seq_length"]
dtype = None
load_in_4bit = True  # 4bit量化节省显存

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Qwen2.5-7B",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)


# ===================== 3. 添加金融场景适配的LoRA适配器 =====================
model = FastLanguageModel.get_peft_model(
    model,
    r=finance_sft_config["lora_r"],
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                   "gate_proj", "up_proj", "down_proj"],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",  # 节省显存
    random_state=3407,
    use_rslora=False,
    loftq_config=None,
)


# ===================== 4. 加载金融SFT数据集（包含来源信息） =====================
qa_data_path = "./datas/finance_sft_train.json"
with open(qa_data_path, "r", encoding="utf-8") as f:
    qa_data = json.load(f)

# 转换为Hugging Face Dataset
dataset = Dataset.from_list(qa_data)
print(f"金融SFT样本数: {len(dataset)}")
print("示例金融样本:", dataset[0])


# ===================== 5. 格式化数据（包含来源信息用于损失计算） =====================
EOS_TOKEN = tokenizer.eos_token
finance_prompt = """你是专业金融分析师助手，需基于研报、财报等金融文档内容回答，严格区分事实与观点，遵循金融术语规范。
请同时输出信息来源（格式：来源: 文件名+页码）。

### 指令:
{instruction}

### 输出:
{output}
来源: {source}"""  # 新增来源字段

def formatting_finance_prompts(examples):
    texts = []
    sources = []  # 单独保存真实来源用于损失计算
    for inst, out, src in zip(examples["instruction"], examples["output"], examples["source"]):
        formatted = finance_prompt.format(
            instruction=inst,
            output=out,
            source=src
        ) + EOS_TOKEN
        texts.append(formatted)
        sources.append(src)
    return {"text": texts, "true_source": sources}

dataset = dataset.map(formatting_finance_prompts, batched=True)


# ===================== 6. 自定义来源一致性损失函数 =====================
def source_consistency_loss(predicted_source, true_source, device):
    """对比预测来源与真实来源的差异（文件名+页码）"""
    if predicted_source != true_source:
        return torch.tensor(0.5, device=device)  # 来源不一致时添加惩罚
    return torch.tensor(0.0, device=device)


# ===================== 7. 重写训练循环集成自定义损失 =====================
class CustomSFTTrainer(SFTTrainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        # 获取原始语言模型损失
        outputs = model(** inputs)
        lm_loss = outputs.loss

        # 提取真实来源
        true_sources = inputs.pop("true_source", None)
        if true_sources is None:
            return (lm_loss, outputs) if return_outputs else lm_loss

        # 解码预测结果以提取来源信息
        device = lm_loss.device
        input_ids = inputs["input_ids"]
        pred_ids = model.generate(
            input_ids=input_ids,
            max_new_tokens=128,
            use_cache=True,
            pad_token_id=tokenizer.pad_token_id
        )
        pred_texts = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)

        # 计算来源一致性损失
        total_source_loss = torch.tensor(0.0, device=device)
        for pred_text, true_src in zip(pred_texts, true_sources):
            # 提取预测中的来源信息（匹配"来源: "后的内容）
            src_match = re.search(r"来源: (.*)", pred_text)
            pred_src = src_match.group(1).strip() if src_match else ""

            # 累加损失
            total_source_loss += source_consistency_loss(pred_src, true_src, device)

        # 计算平均来源损失
        source_loss = total_source_loss / len(true_sources)

        # 总损失 = 语言模型损失 + 来源一致性损失
        total_loss = lm_loss + source_loss

        return (total_loss, outputs) if return_outputs else total_loss


# ===================== 8. 初始化金融场景SFT训练器 =====================
trainer = CustomSFTTrainer(  # 使用自定义训练器
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    packing=False,
    args=SFTConfig(
        per_device_train_batch_size=finance_sft_config["per_device_train_batch_size"],
        gradient_accumulation_steps=finance_sft_config["gradient_accumulation_steps"],
        num_train_epochs=finance_sft_config["num_train_epochs"],
        learning_rate=finance_sft_config["learning_rate"],
        warmup_steps=10,
        max_steps=-1,  # 由num_train_epochs控制
        logging_steps=10,  # 调整日志步长
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="cosine",
        seed=3407,
        output_dir="spark_finance_model_outputs",
        fp16=True,  # 启用混合精度训练
        report_to="none",
    ),
)


# ===================== 9. 显存信息监控 =====================
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU型号: {gpu_stats.name}，最大显存: {max_memory} GB")
print(f"已预留显存: {start_gpu_memory} GB")


# ===================== 10. 启动金融场景模型训练 =====================
trainer_stats = trainer.train()
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"训练耗时: {trainer_stats.metrics['train_runtime']} 秒")
print(f"训练耗时: {round(trainer_stats.metrics['train_runtime']/60, 2)} 分钟")
print(f"训练期间峰值显存: {used_memory} GB，占最大显存 {used_percentage}%")
print(f"LoRA训练显存: {used_memory_for_lora} GB，占最大显存 {lora_percentage}%")


# ===================== 11. 跨集验证：纯新数据评估 =====================
val_data_path = "./datas/finance_sft_val.json"
if os.path.exists(val_data_path):
    with open(val_data_path, "r", encoding="utf-8") as f:
        val_data = json.load(f)
    val_dataset = Dataset.from_list(val_data)
    print(f"\n纯新验证集样本数: {len(val_dataset)}")
    print("验证集示例样本:", val_dataset[0])

    # 格式化验证集数据
    def format_val_prompts(examples):
        texts = []
        labels = []
        sources = []
        for inst, out, src in zip(examples["instruction"], examples["output"], examples["source"]):
            formatted = finance_prompt.format(
                instruction=inst,
                output="",
                source=""
            ) + EOS_TOKEN
            texts.append(formatted)
            labels.append(out)
            sources.append(src)
        return {"text": texts, "label": labels, "true_source": sources}

    val_dataset = val_dataset.map(format_val_prompts, batched=True)

    # 推理并评估
    all_preds = []
    all_labels = []
    all_source_matches = []
    for idx in range(len(val_dataset)):
        prompt = val_dataset[idx]["text"]
        label = val_dataset[idx]["label"]
        true_src = val_dataset[idx]["true_source"]

        input_ids = tokenizer([prompt], return_tensors="pt").to("cuda")
        outputs = model.generate(**input_ids, max_new_tokens=128, use_cache=True)
        pred = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0].strip()

        # 内容匹配判断
        pred_match = 1 if label in pred else 0
        all_preds.append(pred_match)
        all_labels.append(1)

        # 来源匹配判断
        src_match = re.search(r"来源: (.*)", pred)
        pred_src = src_match.group(1).strip() if src_match else ""
        all_source_matches.append(1 if pred_src == true_src else 0)

    # 计算指标
    recall = recall_score(all_labels, all_preds)
    source_acc = np.mean(all_source_matches)
    print(f"\n跨集验证Recall: {recall:.4f}")
    print(f"来源预测准确率: {source_acc:.4f}")
else:
    print("\n警告：未找到纯新验证集文件，请准备finance_sft_val.json后重新运行验证步骤")


# ===================== 12. 金融场景推理与模型保存 =====================
FastLanguageModel.for_inference(model)  # 开启高效推理模式

# 测试金融问答能力（含来源）
test_instruction = "联邦制药2024年营业收入和净利润分别是多少？"
test_prompt = finance_prompt.format(
    instruction=test_instruction,
    output="",
    source=""
)
input_ids = tokenizer([test_prompt], return_tensors="pt").to("cuda")
outputs = model.generate(** input_ids, max_new_tokens=128, use_cache=True)
print("金融问答推理结果:", tokenizer.batch_decode(outputs, skip_special_tokens=True))

# 保存模型
model.save_pretrained("finance_lora_model")
tokenizer.save_pretrained("finance_lora_model")
print("模型保存至 finance_lora_model 目录")